In [ ]:
!pip install Biopython

In [ ]:
import pandas as pd
# get unique species gene name (NCBI ID as well)

# Read the CSV file into a DataFrame
df = pd.read_csv('./domestic_variants_missense_only_purge_3.csv')  # Replace 'your_file.csv' with the actual file path

# Drop duplicates based on 'Species Name' and 'Gene', keeping the first occurrence
unique_df = df.drop_duplicates(subset=['Species Name', 'Gene'], keep='first')

# Select only the relevant columns
result_df = unique_df[['Species Name', 'Gene', 'NCBI ID']]

# Save or display the result
print(result_df)
result_df.to_csv('./unique_species_gene.csv', index=False)  # 

In [ ]:
import pandas as pd
import requests
import time
import csv
# use unique species gene name, get UniProt ID for Human and Animal

# Function to query UniProt for a specific gene and organism
def fetch_uniprot_id(gene, taxonomy_id):
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"gene:{gene} AND taxonomy_id:{taxonomy_id}",
        "format": "tsv",
        "fields": "accession,gene_primary"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200 and response.text:
        lines = response.text.strip().split("\n")
        # Skip header and return the first result's ID
        return lines[1].split("\t")[0] if len(lines) > 1 else "Not Found"
    else:
        return "Not Found"


# Define a mapping dictionary for species name to taxonomy ID
#animal_taxonomy_id = "9986"  # Sheep"9940" # Dog"9615" pig "9823"  cattle "9913"  cat "9685" horse "9796" rabbit "9986"
taxonomy_mapping = {
    'rabbit': "9986",
    'pig': "9823",
    'horse':"9796",
    'cat':"9685",
    'cattle':"9913",
    'dog':"9615",
    'sheep':"9940",
    'goat':"9925",
    # Add more species and their taxonomy IDs as needed
}
# Read the CSV file into a DataFrame
df = pd.read_csv('./unique_species_gene.csv')  # 



# Add a new column 'animal_taxonomy_id' based on the mapping
df['animal_taxonomy_id'] = df['Species Name'].map(taxonomy_mapping)

# Save or display the result
#print(df)
df.to_csv('species_gene_animal_taxonomy_id.csv', index=False)  # Uncomment to save the result to a new CSV file


# Fetch UniProt IDs for human
# Drop duplicates based on 'Gene', this is for Human Uniprot
temp_df = df[['Gene']]
unique_human_gene_df = temp_df.drop_duplicates(subset=['Gene'])
human_gene_list = unique_human_gene_df.Gene.tolist()
human_taxonomy_id = "9606"  # Human
human_uniprot_id_data = []

for gene in human_gene_list:
    human_id = fetch_uniprot_id(gene, human_taxonomy_id)
    print(human_id)
    human_uniprot_id_data.append({"Gene": gene, "Human ID": human_id})

# Create a DataFrame and save as csv
human_uniprot_id_df = pd.DataFrame(human_uniprot_id_data)
print(human_uniprot_id_df)
human_uniprot_id_df.to_csv('./human_uniprot_id.csv', index=False)  # 


# Fetch UniProt IDs for animals 
# Iterate through each row and extract the required columns
animal_uniprot_id_data = []
for index, row in df.iterrows():
    gene = row['Gene']
    ncbi_id = row['NCBI ID']
    animal_taxonomy_id = row['animal_taxonomy_id']
    animal = row['Species Name']
    animal_id = fetch_uniprot_id(gene, animal_taxonomy_id)
    print(animal_id)
    animal_uniprot_id_data.append({"Species Name":animal,"Gene": gene, "Animal ID": animal_id})
    
# Create a DataFrame and save as csv
animal_uniprot_id_df = pd.DataFrame(animal_uniprot_id_data)
print(animal_uniprot_id_df)
animal_uniprot_id_df.to_csv('./animal_uniprot_id.csv', index=False)  # 



In [ ]:
# merge UniProt ID 
import pandas as pd

# Load the CSV files into dataframes
file1 = "./domestic_variants_missense_only_purge_3.csv"   
file2 = "./animal_uniprot_id.csv"  
file3 = "./human_uniprot_id.csv"  

# Assuming tab-delimited files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Merge animal Uniprot ID
merged_df = pd.merge(df1, df2, on=["Species Name", "Gene"], how="inner")
#print(merged_df)

# Merge Human Uniprot ID
merged_df = pd.merge(merged_df, df3, on=[ "Gene"], how="inner")

print(merged_df)

# Save the merged data to a new CSV
output_file = "Uniport_merged_file.csv"
merged_df.to_csv(output_file)

print(f"Merged file saved as {output_file}")

In [ ]:
# get protein sequence with NCBI and Uniport's ID
from Bio import Entrez
from IPython import get_ipython
from IPython.display import display
import requests
import os
from zipfile import ZipFile

def fetch_and_save_protein_sequence_from_ncbi(protein_ids, output_folder):
    # Email required by NCBI to use their API
    Entrez.email = "your_email@example.com"

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List to keep track of filenames for zipping
    file_paths = []

    # Iterate over protein IDs and fetch sequence in FASTA format
    for protein_id in protein_ids:
        
        if protein_id != "Not Found" and protein_id != "":
        
            try:
                # Fetch the record from NCBI using the Entrez module
                handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
                sequence = handle.read()
                handle.close()

                # Define the file path
                file_path = os.path.join(output_folder, f"{protein_id}.txt")
                file_paths.append(file_path)

                # Save the FASTA sequence to the file
                with open(file_path, "w") as file:
                    file.write(sequence)
                print(f"Saved FASTA sequence for {protein_id} to {file_path}")

            except Exception as e:
                print(f"Error fetching sequence for {protein_id}: {e}")




def fetch_and_save_protein_sequence_from_UniProt(protein_ids, output_folder):
    base_url = "https://www.uniprot.org/uniprot/"
    headers = {"Content-Type": "text/plain"}

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List to keep track of filenames for zipping
    file_paths = []

    # Iterate over protein IDs and fetch sequence in FASTA format
    for item in protein_ids:
        # get UniProt ID
        protein_id = item

        if protein_id != "Not Found" and protein_id != "":

          # Construct the URL for the given protein ID
          url = f"{base_url}{protein_id}.fasta"
          response = requests.get(url, headers=headers)
          file_path = os.path.join(output_folder, f"{protein_id}.txt")
          file_paths.append(file_path)

          # Save the FASTA sequence to the file
          with open(file_path, "w") as file:
             file.write(response.text)
             print(f"Saved FASTA sequence for {protein_id} to {file_path}")



#Main
# Load the CSV files into dataframes
file1 = "./animal_uniprot_id.csv"  
file2 = "./human_uniprot_id.csv"  
file3 = "./Uniport_merged_file.csv" 

# Assuming tab-delimited files
df_Animal_ID = pd.read_csv(file1)
df_Human_ID = pd.read_csv(file2)
df_NCBI_ID = pd.read_csv(file3) 


human_ids = df_Human_ID['Human ID'].tolist()
animal_ids = df_Animal_ID['Animal ID'].tolist()
NCBI_ids = df_NCBI_ID['NCBI ID'].tolist()
NCBI_ids = list(set(NCBI_ids)) # ensure the contents of a Python list are unique

#Fetch fasta file fron UniProt
fetch_and_save_protein_sequence_from_UniProt(human_ids, "Human_Uniprot")
fetch_and_save_protein_sequence_from_UniProt(animal_ids, "Animal_Uniprot")
fetch_and_save_protein_sequence_from_ncbi(NCBI_ids, "Animal_NCBI")

In [ ]:
#Verify by search mutant in animal sequence
#some records have no NCBI,no Uniprot
#mutant may not match sequence
#this records will be discarded

In [ ]:
# partial Alignment and calculate similarity percentage

import os
from Bio import SeqIO
from Bio.Align import PairwiseAligner
import re
from Bio import pairwise2
from Bio.Seq import Seq
import pandas as pd

# Function to read FASTA file and return sequence
def read_fasta(file_path):
    with open(file_path, "r") as file:
        seq_record = SeqIO.read(file, "fasta")
        return str(seq_record.seq)

# Function to calculate similarity percentage
def calculate_similarity(aligned_seq1, aligned_seq2):
    matches = sum(1 for a, b in zip(aligned_seq1, aligned_seq2) if a == b)
    total = len(aligned_seq1)
    return (matches / total) * 100


# Function to calculate similarity percentage for specified fragments
def calculate_fragment_similarity(aligned_seq1, aligned_seq2, position, window):
    """
    Extracts a fragment of length 2*window centered at the specified position in the aligned sequences,
    and calculates the similarity percentage.

    Args:
        aligned_seq1 (str): The first aligned protein sequence.
        aligned_seq2 (str): The second aligned protein sequence.
        position (int): The 1-based index of the amino acid around which the fragment is extracted.
        window (int): The number of amino acids before and after the position to include (default is 20).

    Returns:
        float: The similarity percentage between the fragments.
    """
    # Convert 1-based position to 0-based index for slicing
    index = position - 1

    # Extract the fragments around the specified position
    start = max(0, index - window)
    end = index + window + 1

    fragment1 = aligned_seq1[start:end]
    fragment2 = aligned_seq2[start:end]

    # Log a message if the fragment is shorter than expected
    if len(fragment1) < 2 * window + 1 or len(fragment2) < 2 * window + 1:
        print(f"Warning: Fragment length is shorter than expected at position {position}.\n"
              f"Fragment1 length: {len(fragment1)}, Fragment2 length: {len(fragment2)}")

    # Calculate similarity percentage
    matches = sum(1 for a, b in zip(fragment1, fragment2) if a == b)
    total = len(fragment1)
    similarity = (matches / total) * 100

    return similarity

def extract_and_convert(mutation_string):
  """Extracts the numerical part of a mutation string and converts it to an integer.

  Args:
    mutation_string: The input mutation string (e.g., "R301C").

  Returns:
    The numerical part as an integer, e.g., 301, or None if not found.
  """
  match = re.search(r"(\d+)", mutation_string)  # Find the numerical part
  if match:
    print( int(match.group(1)))
    return int(match.group(1))  # Convert to integer
  else:
    return None


def check_mutation(sequence, mutation):
    """
    Checks if the specified amino acid mutation matches the sequence in the FASTA file.

    :param fasta_file: Path to the FASTA file.
    :param mutation: Mutation string in the format 'M200T'.
    :return: True if the amino acid matches, otherwise False.
    """
    # Parse the mutation
    original_aa = mutation[0]
    position = int(mutation[1:-1])
    mutated_aa = mutation[-1]


    # Check if the amino acid at the specified position matches
    if position <= len(sequence) and sequence[position - 1] == original_aa:
        print(f"Position {position} in the sequence matches the original amino acid '{original_aa}'.")
        return True
    else:
        print(f"Mismatch: Position {position} does not have the amino acid '{original_aa}' in the sequence.")
        return False



# Function to format sequences with highlighted positions
# Problem: it seems some animal mutants do not match corresponding NCBI sequence, in this case, will try sequence form UniProt, if fails again, return Non
# it there is no match human amino acid, rerurn -1 and skip without output
def format_sequence_with_positions(human_sequence, horse_sequence, animal_mutant):
    """Formats sequences with residue numbering and highlights multiple residues in both sequences.

    Args:
        human_sequence: The aligned human protein sequence.
        horse_sequence: The aligned horse protein sequence.
        horse_residue_positions: A list of positions to highlight in the horse sequence.

    Returns:
        A formatted string with residue numbering and highlighting.
    """
    formatted = ""
    human_residue_number = 1
    horse_residue_number = 1
    horse_mutant = ""
    humnan_mutant = ""
    alignment_sum = ""


    #print(animal_mutant) pass
    # R301C to 301
    horse_residue_positions = extract_and_convert(animal_mutant)   # Convert to integers


    for i in range(0, len(human_sequence), 100):
        human_line = human_sequence[i:i+100]
        horse_line = horse_sequence[i:i+100]

        human_line_numbered = f"{human_residue_number:>5} "
        horse_line_numbered = f"{horse_residue_number:>5} "
        match_line = " " * 5

        for h_residue, hs_residue in zip(human_line, horse_line):
            
            if h_residue != '-':
                # Highlight corresponding human residues with []
                if horse_residue_number == horse_residue_positions:
                    human_line_numbered += f"[{h_residue}]"
                    humnan_mutant = h_residue # save
                else:
                    human_line_numbered += h_residue
                human_residue_number += 1
            else:
                # no matching human residue
                human_line_numbered += " "
                #print(f"No matching human residue with mutant {animal_mutant}")
                #result_file.write(f"{animal} {gene} | No matching human residue found for mutant {animal_mutant} {e}\n")
                #continue #skip to next record

            if hs_residue != '-':
                # Highlight specified horse residues with []
                if horse_residue_number == horse_residue_positions:
                    horse_line_numbered += f"[{hs_residue}]"
                    horse_mutant = hs_residue # save
                    human_pos = find_corresponding_human_pos(human_sequence, horse_sequence, horse_residue_number)
                    formatted += f"Animal residue position {horse_mutant}{horse_residue_number} corresponds to Human residue position {humnan_mutant}{human_pos}\n"
                    alignment_sum += f"Animal mutant {animal_mutant} : Human mutant {humnan_mutant}{human_pos}{animal_mutant[-1]}\n"
                else:
                    horse_line_numbered += hs_residue
                horse_residue_number += 1
            else:
                horse_line_numbered += " "

            match_line += "*" if h_residue == hs_residue and h_residue != '-' else " "

        formatted += f"Animal:  {horse_line_numbered}\n"
        formatted += f"Human:   {human_line_numbered}\n"
        formatted += f"        {match_line}\n\n"

    formatted += f"{alignment_sum}\n"
    return formatted

# Function to find corresponding human residue position
def find_corresponding_human_pos(human_seq, horse_seq, horse_pos):
    """Finds the corresponding human residue position for a given Animal residue position."""
    horse_count = 0
    human_count = 0

    for i in range(len(horse_seq)):
        if horse_count == horse_pos:
            return human_count

        if horse_seq[i] != '-':
            horse_count += 1
        if human_seq[i] != '-':
            human_count += 1

    return -1  # If not found (shouldn't happen in normal cases)

# Main function to align and generate report
def align_and_generate_report(merged_all_data, output_file="alignment_results.txt", human_data_dir="./Human_Uniprot", horse_data_dir="./Animal_Uniprot",horse_data_NCBI_dir="./Animal_NCBI"):

    aligner = PairwiseAligner()

    with open(output_file, "w") as result_file:
        result_file.write("Human ID | Species Name | Protein Name | Human Sequence | Animal Sequence | Similarity (%)\n")
        result_file.write("="*80 + "\n")


        for index, row in merged_all_data.iterrows():
             animal = row['Species Name']
             gene = row['Gene']
             pathog = row['Deleterious?']
             human_protein_id = row['Human ID']
             horse_protein_id = row['Animal ID']
             horse_NCBI_id = row['NCBI ID']
             animal_mutant = row['Animal Mutant']

             print(animal, gene, animal_mutant)

             try:
                # read human sequence 
                human_file_path = os.path.join(human_data_dir, f"{human_protein_id}.txt")
                if not os.path.exists(human_file_path):
                  raise FileNotFoundError(f"Warning: Human Sequence File not found: {human_file_path}")
                else:
                  human_seq = read_fasta(human_file_path)

                # read animal sequence, uniprot
                horse_file_path = os.path.join(horse_data_dir, f"{horse_protein_id}.txt")
                if not os.path.exists(horse_file_path):
                   print(f"Warning: Animal Sequence (UniProt) File not found: {horse_file_path}")
                   horse_seq = ""
                else:
                  horse_seq = read_fasta(horse_file_path)

                # read animal sequence, NCBI
                horse_NCBI_file_path = os.path.join(horse_data_NCBI_dir, f"{horse_NCBI_id}.txt")
                if not os.path.exists(horse_NCBI_file_path):
                   print(f"Warning: Animal Sequence (NCBI) File not found: {horse_NCBI_file_path}")
                   horse_NCBI_seq = ""
                else:
                  horse_NCBI_seq = read_fasta(horse_NCBI_file_path)


                #check which animal sequence match the mutant
                if check_mutation(horse_NCBI_seq, animal_mutant):
                  print("Using NCBI Sequence File for alignment!")
                  alignment = aligner.align(human_seq, horse_NCBI_seq)
                else:
                  if check_mutation(horse_seq, animal_mutant):
                    print("Using UniProt Sequence File for alignment!")
                    alignment = aligner.align(human_seq, horse_seq)
                  else:
                    print(f"No Sequence File matchs {gene} with mutant {animal_mutant}")
                    result_file.write(f"{animal} {gene} | No matching sequence found for mutant {animal_mutant} due to no matching animal sequence {e}\n")
                    continue #skip to next record


                aligned_human = str(alignment[0][0])
                aligned_horse = str(alignment[0][1])

                similarity = calculate_similarity(aligned_human, aligned_horse)
                 
                horse_residue_positions = extract_and_convert(animal_mutant)   #   Specify the 1-based position of the amino acid
                similarity20 = calculate_fragment_similarity(aligned_horse, aligned_human, horse_residue_positions,10)
                similarity40 = calculate_fragment_similarity(aligned_horse, aligned_human, horse_residue_positions,20)
                similarity60 = calculate_fragment_similarity(aligned_horse, aligned_human, horse_residue_positions,30)
                similarity80 = calculate_fragment_similarity(aligned_horse, aligned_human, horse_residue_positions,40)
                #print(similarity) pass

                #print(horse_residue_positions) pass

                formatted_sequences = format_sequence_with_positions(aligned_human, aligned_horse, animal_mutant)
                result_file.write(f"{human_protein_id} |") 
                result_file.write(f"{animal} |")
                result_file.write(f"{pathog} |")
                result_file.write(f"{gene} |\n{formatted_sequences}\n")
                result_file.write(f"Similarity: {similarity:.2f}%\n")
                result_file.write(f"Similarity20: {similarity20:.2f}%\n")
                result_file.write(f"Similarity40: {similarity40:.2f}%\n")
                result_file.write(f"Similarity60: {similarity60:.2f}%\n")
                result_file.write(f"Similarity80: {similarity80:.2f}%\n")
                result_file.write("="*50 + "\n")

                print(f"Alignment for {gene} completed and written to file.")
             except Exception as e:
                print(f"Error processing {gene}: {e}")
                result_file.write(f"{gene} | Error: {e}\n")
                result_file.write("="*50 + "\n")



# Load merged_all_data
all_data = pd.read_csv("./Uniport_merged_file.csv")

# Run the main function
align_and_generate_report(all_data)


In [16]:
def summarize_alignment(input_file, output_file):
  """Reads an alignment file, removes lines starting with "Animal:", "Human:", "Horse residue position",
  containing "*", and blank lines, and writes the remaining lines to a new file.

  Args:
    input_file: Path to the input alignment file.
    output_file: Path to the output summary file.
  """
  try:
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if not line.startswith(('Animal:', 'Human:', 'Animal residue position')) and "*" not in line and line.strip():
                outfile.write(line)
    print(f"Alignment summary written to {output_file}")
  except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")

# Example usage:
input_file = "alignment_results.txt"
output_file = "alignment_summary.txt"
summarize_alignment(input_file, output_file)

Alignment summary written to alignment_summary.txt


In [ ]:
# combine all data for prediction
import pandas as pd

# Read the file
file_path = "alignment_summary.txt"  # Replace with your file path
with open(file_path, "r") as file:
    lines = file.readlines()

# Initialize lists to hold data
human_id = []
animal = []
pathog = []
Gene = []
Animal_mutant = []
Human_mutant = []
similarities = []
similarities20 = []
similarities40 = []
similarities60 = []
similarities80 = []

# Parse the lines, skipping the first line (header)
for line in lines[1:]:  # Start from the second line
    line = line.strip()
    if line.endswith("|") and not line.startswith("="):
        human_id.append(line.split(" |")[0].strip())
        animal.append(line.split(" |")[1].strip())
        pathog.append(line.split(" |")[2].strip())
        Gene.append(line.split(" |")[3].strip())
        # Ensure all other lists grow as well
        Animal_mutant.append(None)
        Human_mutant.append(None)
        similarities.append(None)
        similarities20.append(None)
        similarities40.append(None)
        similarities60.append(None)
        similarities80.append(None)
    elif "Animal mutant" in line:
        parts = line.split(":")
        Animal_mutant[-1] = parts[0].split("Animal mutant")[-1].strip()
        Human_mutant[-1] = parts[1].split("Human mutant")[-1].strip()
    elif "Similarity" in line and "Similarity20" not in line and "Similarity40" not in line and "Similarity60" not in line and "Similarity80" not in line:
        similarities[-1] = float(line.split(":")[-1].strip().replace("%", ""))
    elif "Similarity20" in line:
        similarities20[-1] = float(line.split(":")[-1].strip().replace("%", ""))
    elif "Similarity40" in line:
        similarities40[-1] = float(line.split(":")[-1].strip().replace("%", ""))
    elif "Similarity60" in line:
        similarities60[-1] = float(line.split(":")[-1].strip().replace("%", ""))
    elif "Similarity80" in line:
        similarities80[-1] = float(line.split(":")[-1].strip().replace("%", ""))

# Create the DataFrame
data = {
    "Human ID":human_id,
    "Species Name":animal,
    "Gene": Gene,
    "Animal Mutant": Animal_mutant,
    "'Deleterious?":pathog,
    "Human Mutant": Human_mutant,
    "Similarity (%)": similarities,
    "Similarity20 (%)": similarities20,
    "Similarity40 (%)": similarities40,
    "Similarity60 (%)": similarities60,
    "Similarity80 (%)": similarities80,
}

#print(data)

df = pd.DataFrame(data)

# Label records which have no matching human amino acid
df['Label'] = df['Human Mutant'].apply(lambda x: "No Matching" if x[0].isdigit() else "")

print(df)
 
#save All_data as csv file
df.to_csv('final_all_data.csv')
print("Final data saved to final_all_data.csv")


# Discard rows where the "Label" column is "No Matching"
df_filtered = df[df['Label'] != 'No Matching']
# save work data
df_filtered.to_csv('work_all_data.csv')
print("Working data saved to work_all_data.csv")

            ####################################################################################
            ####################################################################################
            ####################################################################################
                            Follwing code is for ESM prediction
            ####################################################################################
            ####################################################################################
            ####################################################################################

In [17]:
import pandas as pd
import os
import shutil
# selected needed LLR files by Human UniProt ID and save at ./selected_LLR

# Path to the directory containing the CSV files
directory_path = '../ESM1b/LLR/content'

# Path to the destination directory where selected files will be copied
destination_path = '../ESM1b/selected_LLR'

# Read the list CSV file
list_df = pd.read_csv('./human_uniprot_id.csv')  # Adjust the path to the uploaded file

# Extract the UniProt codes from the list
uniprot_id = list_df['Human ID'].tolist()

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Iterate through the directory and select files
for filename in os.listdir(directory_path):
    # Check if the filename matches the exact pattern '<uniprot_id>_LLR.csv'
    if any(filename == f"{id}_LLR.csv" for id in uniprot_id):
        source_file = os.path.join(directory_path, filename)
        destination_file = os.path.join(destination_path, filename)
        shutil.copyfile(source_file, destination_file)

print("Selected ESM LLR files have been copied successfully.")

Selected ESM LLR files have been copied successfully.


In [ ]:
# transfer original ESM LLR data to format of AlphaMisence
import pandas as pd
import os

# 	M 1	G 2
# K	-14.012	-6.545
# R	-11.862	-5.078
# H	-14.163	-7.058
# E	-12.91	-5.521
# to
# M1K	-14.012
# G2K	-6.545
# S3K	-6.324
# R4K	-6.808
# C5K	-3.984
# A6K	-8.685
# L7K	-9.151
# transfer all selected LLR files to new data format

directory_path = '../ESM1b/selected_LLR'

# Path to the destination directory where selected files will be copied
output_directory = '../ESM1b/selected_transferred_LLR'

# Read the list CSV file
list_df = pd.read_csv('./human_uniprot_id.csv') 

# Extract the UniProt codes from the list
uniprot_id = list_df['Human ID'].tolist()

# Iterate through the directory and select files
for filename in uniprot_id:
    source_file = os.path.join(directory_path, filename + '_LLR.csv')

    if os.path.exists(source_file):

        df = pd.read_csv(source_file, index_col=0)

        # Create a list to store the new rows
        new_rows = []

        # Iterate through the dataframe and construct the new rows
        for index, row in df.iterrows():
            for col in df.columns:
                new_row = {'Key': f"{col}{index}", 'Value': row[col]}
                new_rows.append(new_row)

        # Create a new dataframe from the new rows
        new_df = pd.DataFrame(new_rows)

        # Remove all blanks from the 'Key' column
        new_df['Key'] = new_df['Key'].str.replace(' ', '')

        # Save the new dataframe to a CSV file
        output_file = os.path.join(output_directory, filename + '_LLR.csv' )
        new_df.to_csv(output_file, index=False, header=False)

        print(filename + '_LLR.csv' + " has been created successfully.")
    else:
        print(filename + '_LLR.csv' + ' not exists.')

In [19]:
import pandas as pd
# search each mutant in LLR file and get score
# Step 1: Read the input CSV file

df1 = pd.read_csv('work_all_data.csv')  # Adjust the path to the uploaded file


# Folder path for the alpha CSV files
folder_path = '../ESM1b/selected_transferred_LLR'

# Columns to append
df1['ESM_Score'] = None

# Iterate through df1 rows
for index, row in df1.iterrows():
    human_id = row['Human ID']
    human_mutant = row['Human Mutant']

    # Construct file path for the corresponding CSV file
    filename = human_id + '_LLR.csv'
    file_path = os.path.join(folder_path, filename)

    if os.path.exists(file_path):
        # Load the CSV file
        csv_data = pd.read_csv(file_path, header=None)
        headers = ['Mutant', 'ESM_Value']
        csv_data.columns = headers
        #print (csv_data)
        # Search for the Human Mutant in the protein_variant column
        match = csv_data[csv_data['Mutant'] == human_mutant]
        #print (match)
        if not match.empty:
            # Retrieve am_pathogenicity and am_class values
            df1.at[index, 'ESM_Score'] = match['ESM_Value'].values[0]

# Save the updated df1 to a new file
df1.to_csv("work_all_data_ESM.csv", index=False)

print("Done ESM Score")

Done ESM Score


            ####################################################################################
            ####################################################################################
            ####################################################################################
                    The following code is for AlphaMisense prediction
            ####################################################################################
            ####################################################################################
            ####################################################################################

In [ ]:
#get alphamissence scroe data in csv format, each human protein with one csv file
#https://alphafold.ebi.ac.uk/files/AF-Q01726-F1-hg19.csv
#https://alphafold.ebi.ac.uk/files/AF-Q01726-F1-hg38.csv
# get alphamissence score data from alphafold.ebi.ac.uk
#https://alphafold.ebi.ac.uk/files/AF-Q9H2P0-F1-aa-substitutions.csv

import os
import requests
import time
import pandas as pd



All_data_df = pd.read_csv("./human_uniprot_id.csv")

# List of gene IDs
gene_ids = All_data_df['Human ID'].tolist() #    ["Q01726", "Q04526"]   Add your gene IDs here

# Folder to save the downloaded files
output_folder = "../AlphaMissense/Alpha_substitutions"
os.makedirs(output_folder, exist_ok=True)

# Base URL for downloading files
base_url = "https://alphafold.ebi.ac.uk/files/"

# Download files with a 0.1-second delay
for gene_id in gene_ids:
    #file_name = f"AF-{gene_id}-F1-hg38.csv"
    #https://alphafold.ebi.ac.uk/files/AF-Q9H2P0-F1-aa-substitutions.csv
    file_name = f"AF-{gene_id}-F1-aa-substitutions.csv"
    url = f"{base_url}{file_name}"
    output_file = os.path.join(output_folder, f"{gene_id}.csv")

    try:
        print(f"Downloading: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for failed requests

        # Save the file with the new name
        with open(output_file, "wb") as file:
            file.write(response.content)
        print(f"Saved: {output_file}")
    except requests.RequestException as e:
        print(f"Failed to download {url}: {e}")

    # Wait for 0.05 second before sending the next request
    time.sleep(0.05)

In [11]:
#alphamissence score predict
# All data from alignment
#	Human ID	Species Name	Gene	Animal Mutant	Human Mutant	Similarity (%)	Similarity20 (%)	Similarity40 (%)	Similarity60 (%)	Similarity80 (%)	Label
#0	Q9UPW5      sheep	       AGTPBP1	    R970P	      R970P	           91.19	         100	            95.12	           96.72	             97.53	


import os
import pandas as pd

# Load the work data, 15 redords without matching human residues have beed discarded 
df1 = pd.read_csv("./work_all_data.csv")

# Folder path for the alpha CSV files
folder_path = "../AlphaMissense/Alpha_substitutions"

# Columns to append
df1['am_pathogenicity'] = None
df1['am_class'] = None

# Iterate through df1 rows
for index, row in df1.iterrows():
    human_id = row['Human ID']
    human_mutant = row['Human Mutant']

    # Construct file path for the corresponding CSV file
    file_path = os.path.join(folder_path, f"{human_id}.csv")

    if os.path.exists(file_path):
        # Load the CSV file
        csv_data = pd.read_csv(file_path)

        # Search for the Human Mutant in the protein_variant column
        match = csv_data[csv_data['protein_variant'] == human_mutant]

        if not match.empty:
            # Retrieve am_pathogenicity and am_class values
            df1.at[index, 'am_pathogenicity'] = match['am_pathogenicity'].values[0]
            df1.at[index, 'am_class'] = match['am_class'].values[0]

# Save the updated df1 to a new file
df1.to_csv("work_all_data_alphamissense.csv", index=False)

print("Updated df1 saved as 'work_all_data_alphamissense.csv'.")

Updated df1 saved as 'work_all_data_alphamissense.csv'.


In [ ]:

PolyPhen-2



In [20]:
import csv
# prepare polyphen-2 input data

# Input and output file paths
input_file = "./work_all_data.csv" 
output_file = './polyphen_input.txt'

# Open the CSV file and read its contents
with open(input_file, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # Open the output file in write mode
    with open(output_file, mode='w') as txt_file:
        for row in csv_reader:
            human_id = row['Human ID']
            position = ''.join(filter(str.isdigit, row['Human Mutant']))
            original = row['Human Mutant'][0]
            mutant = row['Human Mutant'][-1]
            txt_file.write(f"{human_id} {position} {original} {mutant}\n")

print(f"Data successfully written to {output_file} in polyphen-2 input format")

Data successfully written to ./polyphen_input.txt in polyphen-2 input format


In [ ]:
# transfer polyphen-2 result to csv format 

import csv

# Input and output file paths
input_file = './pph2-short.txt'  # Replace with your TXT file path
output_file = './polyphen2_output.csv'

# Open the TXT file and read its contents
with open(input_file, mode='r') as txt_file:
    # Read and clean the header
    headers = txt_file.readline().strip().split('\t')
    headers = [header.strip() for header in headers]  # Remove extra spaces

    # Debugging: Print the headers to verify
    print("Headers in TXT file:", headers)

    # Identify the indexes of needed columns
    columns_to_keep = ['#o_acc', 'o_pos', 'o_aa1', 'o_aa2', 'prediction', 'pph2_prob']
    try:
        column_indexes = [headers.index(col) for col in columns_to_keep]
    except ValueError as e:
        print(f"Error: One or more columns from {columns_to_keep} are missing in the file headers.")
        raise e

    # Open the CSV file in write mode
    with open(output_file, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Write the header row
        csv_writer.writerow(['Human ID', 'Human Mutant', 'Prediction', 'PPH2_Prob'])

        # Process each line in the TXT file
        for line_number, line in enumerate(txt_file, start=2):  # Start with line 2 for debugging
            row = line.strip().split('\t')

            # Skip rows with insufficient columns
            if len(row) < len(headers):
                print(f"Warning: Row {line_number} has fewer columns than expected. Skipping.")
                continue

            try:
                # Extract data using column indexes
                human_id = row[column_indexes[0]]
                position = row[column_indexes[1]]
                aa1 = row[column_indexes[2]]
                aa2 = row[column_indexes[3]]
                prediction = row[column_indexes[4]]
                pph2_prob = row[column_indexes[5]]

                # Create the new Human Mutant column in R970P format
                human_mutant = f"{aa1}{position}{aa2}"

                #clean
                human_id = human_id.replace(" ", "")
                human_mutant = human_mutant.replace(" ", "")

                # Write the processed row to the CSV
                csv_writer.writerow([human_id, human_mutant, prediction, pph2_prob])
            except IndexError as e:
                print(f"Error: Issue processing row {line_number}. Row data: {row}")
                raise e

print(f"Data successfully written to {output_file}")

In [36]:
# merge polyphen2 output with working data
import pandas as pd

# Read the first CSV file
file1 = "./work_all_data.csv"    
df1 = pd.read_csv(file1)

# Read the second CSV file
file2 = './polyphen2_output.csv'  
df2 = pd.read_csv(file2)

# Merge the two dataframes on 'Human ID' and 'Human Mutant'
merged_df = pd.merge(df1, df2, on=['Human ID', 'Human Mutant'], how='left')

# Save the merged result to a new CSV file
output_file = "./work_all_data_PolyPhen.csv"    
merged_df.to_csv(output_file, index=False)

print(f"Merged file saved as {output_file}")

Merged file saved as ./work_all_data_PolyPhen.csv
